In [1]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [2]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [3]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [8]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[8.2737e+02, 1.1000e+00],
         [5.1197e+01, 1.1486e-02],
         [1.3739e+02, 1.0875e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[2.2813e+02, 1.1000e+00],
         [6.8024e+01, 2.8529e-01],
         [7.1060e+01, 3.0230e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[4.0124e+02, 1.1000e+00],
         [5.7070e+01, 1.0000e+00],
         [7.1060e+01, 2.0635e-01],
         ...,
         [2.8815e+02, 1.6709e-01],
         [2.8915e+02, 6.6497e-01],
         [3.1719e+02, 4.5205e-01]],

        ...,

        [[4.0920e+02, 1.1000e+00],
         [1.2301e+02, 5.2116e-02],
         [1.6502e+02, 1.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[4.2512e+02, 1.1000e+00],
         [2.1701e+02, 2.

In [9]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 200, 256, 256, 256, 256, 256, 256, 256, 256])


In [10]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [11]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [12]:
tmp_original['precursor_mz']

tensor([827.3680, 228.1277, 401.2435, 564.5360, 279.1591, 836.6170, 503.2450,
        213.1022, 328.1540, 409.2005, 425.1200, 409.1983])

In [13]:
tmp_original['adduct'], 

(['[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+'],)

In [14]:
tmp_original['mol_freq']

tensor([ 63.,  60.,  25.,  27.,  46., 104.,  30., 178.,   3.,   2.,  46.,   3.])

In [15]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [16]:
sum(tmp_original['labels'])

tensor(12)

In [17]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768,  968, 1224, 1480, 1736, 1992, 2248, 2504, 2760])

In [18]:
len(tmp_original['labels'])

3016

In [19]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 200, 256, 256, 256, 256, 256, 256, 256, 256])

In [20]:
sum(tmp_original['batch_ptr'])

tensor(3016)

In [21]:
len(tmp_original['candidates'])

3016

In [22]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [23]:
len(tmp_original['candidates_smiles'])

3016

# MSnRetrieval

In [24]:
from massspecgym.featurize import SpectrumFeaturizer

In [25]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [26]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [27]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [31]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/min_sample_trees.mgf"


In [28]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 16476
MSnRetrievalDataset length: 16476
Using 5 processes for precomputation.


In [29]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [30]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 12536
Val dataset size: 1952


In [31]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

[PROFILE] __getitem__ took 0.474467 seconds
[PROFILE] __getitem__ took 0.026641 seconds
[PROFILE] __getitem__ took 0.022650 seconds
[PROFILE] __getitem__ took 0.023300 seconds
[PROFILE] __getitem__ took 0.022671 seconds
[PROFILE] __getitem__ took 0.023122 seconds
[PROFILE] __getitem__ took 0.020543 seconds
[PROFILE] __getitem__ took 0.020759 seconds
[PROFILE] __getitem__ took 0.022244 seconds
[PROFILE] __getitem__ took 0.021315 seconds
[PROFILE] __getitem__ took 0.020967 seconds
[PROFILE] __getitem__ took 0.020539 seconds
[PROFILE] collate_fn took 0.017718 seconds
{'spec': DataBatch(x=[109, 1039], edge_index=[2, 194], batch=[109], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([538.3011, 311.1278, 427.0110, 485.3136, 327.2067, 538.2424, 444.

In [32]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 141, 256, 256, 256,  27, 256, 256, 256, 256],
       dtype=torch.int32)


In [33]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

Data types in tmp_original:
  spec: <class 'torch.Tensor'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  identifier: <class 'list'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>
Data types in tmp_msn:
  spec: <class 'abc.DataBatch'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  identifier: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [34]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [35]:
tmp_msn['spec'] 

DataBatch(x=[109, 1039], edge_index=[2, 194], batch=[109], ptr=[13])

In [36]:
tmp_msn['precursor_mz']

tensor([538.3011, 311.1278, 427.0110, 485.3136, 327.2067, 538.2424, 444.3108,
        528.1880, 282.1852, 409.0546, 496.2177, 472.1701])

In [37]:
tmp_msn['mol_freq']

tensor([1., 2., 1., 1., 5., 6., 1., 1., 1., 1., 1., 1.])

In [38]:
tmp_msn['identifier']

['0014753_0000000',
 '0004247_0000000',
 '0083725_0000000',
 '0024339_0000000',
 '0042593_0000000',
 '0100134_0000000',
 '0011291_0000000',
 '0093239_0000000',
 '0083263_0000000',
 '0049559_0000000',
 '0000285_0000000',
 '0038412_0000000']

In [39]:
tmp_msn['adduct']

['[M+NH4]+',
 '[M+H-H2O]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+H]+']

In [40]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [41]:
tmp_msn['smiles']

['C/C(=C\\COC(=O)C)/C(CC1C(=C)C(C(=O)C2C1(CCC(C2(C)C)OC(=O)C)C)OC(=O)C)OC(=O)C',
 'CC1CC2=C(C3C(=O)C4=C(C(C3(CC2)O)O)C(=CC=C4)O)C(=O)C1',
 'C1=CC2=C(C=CC3=C2C(=C1)C(=O)N(C3=O)CCN)SC4=CC=C(C=C4)Br',
 'CN1CCCC1=NC(=NC2=CC=CC=C2)N3CCN(CC3)C(=NC4=CC=CC=C4)N=C5CCCN5C',
 'CC[C@H]1CN2CC[C@H]1C[C@H]2[C@@H](C3=C4C=C(C=CC4=NC=C3)OC)O',
 'CCN1CCN(CC1)CC2=C(C=C(C=C2)NC(=O)C3=CC(=C(C=C3)C)OC4=C5C=CNC5=NC=C4)C(F)(F)F',
 'CC1C2C(NC(=O)C23C(/C=C(/CCC4C(CCC3=O)OC(O4)(C)C)\\C)C=C1C)CC(C)C',
 'C1CN(CCN1CC2=CC=C(C=C2)C3=C(C=C(C=C3)C(C(F)(F)F)(C(F)(F)F)O)F)CC4=CC=NC=C4',
 'CN(C)CC1CCOC1(C2=CC=CC=C2)C3=CC=CC=C3',
 'C1=CC=C(C=C1)C(CNC(=O)C2=CC=C(O2)Br)C3=CNC4=CC=CC=C43',
 'C1[C@H]([C@@H]([C@H]([C@@H](O1)O[C@@H](CCC2=CC(=C(C=C2)O)O)CC(=O)CCC3=CC(=C(C=C3)O)O)O)O)O',
 'CC(C)CN1C(=O)C2=C(C=C(C=C2)C(=O)NCCOC)N=C1SCC(=O)C3=CC=C(C=C3)F']

In [42]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [43]:
sum(tmp_msn['labels'])

tensor(12)

In [44]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768,  909, 1165, 1421, 1677, 1704, 1960, 2216, 2472])

In [45]:
len(tmp_msn['labels'])

2728

In [46]:
tmp_msn['batch_ptr']

tensor([256, 256, 256, 141, 256, 256, 256,  27, 256, 256, 256, 256],
       dtype=torch.int32)

In [47]:
sum(tmp_msn['batch_ptr'])

tensor(2728, dtype=torch.int32)

In [48]:
len(tmp_msn['candidates'])

2728

In [49]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [50]:
tmp_msn.keys() == tmp_original.keys()

True

In [51]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [52]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [53]:
tmp_original['batch_ptr'].dtype

torch.int64

In [54]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [55]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2728, dtype=torch.int32), 2728)

In [56]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(3016), 3016)

In [57]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [58]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [59]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [60]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [61]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [62]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [63]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [64]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [65]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [66]:
len(tmp_msn['candidates_smiles'])

2728